In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
%matplotlib inline

In [2]:
import os
import torch
import torch.nn as nn

### 데이터를 불러오자

In [12]:
dir = os.getcwd()
train = pd.read_csv(dir + "/data/preprocessed_data/train_data.csv")
test = pd.read_csv(dir + "/data/preprocessed_data/test_data.csv")
valid = pd.read_csv(dir + "/data/preprocessed_data/valid_data.csv")

### 데이터 확인

In [5]:
train.head()

,cust_no,label,E1,E2,E3,E4,E5,E6,E10,E14,...,농축업,무직,사무원,생산직,서비스직,은퇴,전문직,정치인,판매원,프리랜서
0,0xb2d283b6,1.0,965,965.0,965.0,1,0,209.0,210.0,18169,...,0,0,0,0,0,0,0,0,0,1
1,0xb2d62fab,1.0,368,368.0,368.0,0,1,173.0,275.0,18260,...,0,0,0,0,0,0,0,0,1,0
2,0xb2d69cdb,1.0,199,199.0,199.0,0,1,41.0,6.0,18168,...,0,0,0,0,0,0,0,0,1,0
3,0xb2d942e8,-1.0,120,120.0,120.0,0,1,11.0,1.0,18261,...,0,0,0,0,0,0,0,0,1,0
4,0xb2d9156f,1.0,40,40.0,40.0,0,0,18.0,1.0,18169,...,0,0,0,0,0,0,0,0,1,0


In [6]:
pd.set_option('display.max_rows',None)
train.isnull().sum()

cust_no     0
label       0
E1          0
E2          0
E3          0
E4          0
E5          0
E6          0
E10         0
E14         0
E15         0
E16         0
E17         0
E18         0
I1         54
I2          0
I3          0
I4          0
I6          0
I7          0
I11         0
I15         0
I16         0
I17         0
I18         0
I19         0
I20         0
X1_m1       0
X2_m1       0
X3_m1       0
X4_m1       0
X5_m1       0
X6_m1       0
X7_m1       0
X8_m1       0
B1_m1       0
B2_m1       0
B3_m1       0
B4_m1       0
B5_m1       0
C1_m1       0
C2_m1       0
X1_m2       0
X2_m2       0
X3_m2       0
X4_m2       0
X5_m2       0
X6_m2       0
X7_m2       0
X8_m2       0
B1_m2       0
B2_m2       0
B3_m2       0
B4_m2       0
B5_m2       0
C1_m2       0
C2_m2       0
X1_m3       0
X2_m3       0
X3_m3       0
X4_m3       0
X5_m3       0
X6_m3       0
X7_m3       0
X8_m3       0
B1_m3       0
B2_m3       0
B3_m3       0
B4_m3       0
B5_m3       0
B6_m3       0
B7_m3 

In [7]:
test.isnull().sum()

cust_no    0
label      0
E1         0
E2         0
E3         0
E4         0
E5         0
E6         0
E10        0
E14        0
E15        0
E16        0
E17        0
E18        0
I1         7
I2         0
I3         0
I4         0
I6         0
I7         0
I11        0
I15        0
I16        0
I17        0
I18        0
I19        0
I20        0
X1_m1      0
X2_m1      0
X3_m1      0
X4_m1      0
X5_m1      0
X6_m1      0
X7_m1      0
X8_m1      0
B1_m1      0
B2_m1      0
B3_m1      0
B4_m1      0
B5_m1      0
C1_m1      0
C2_m1      0
X1_m2      0
X2_m2      0
X3_m2      0
X4_m2      0
X5_m2      0
X6_m2      0
X7_m2      0
X8_m2      0
B1_m2      0
B2_m2      0
B3_m2      0
B4_m2      0
B5_m2      0
C1_m2      0
C2_m2      0
X1_m3      0
X2_m3      0
X3_m3      0
X4_m3      0
X5_m3      0
X6_m3      0
X7_m3      0
X8_m3      0
B1_m3      0
B2_m3      0
B3_m3      0
B4_m3      0
B5_m3      0
B6_m3      0
B7_m3      0
C1_m3      0
C2_m3      0
군인         0
농축업        0
무직         0

In [8]:
valid.isnull().sum()

cust_no    0
label      0
E1         0
E2         0
E3         0
E4         0
E5         0
E6         0
E10        0
E14        0
E15        0
E16        0
E17        0
E18        0
I1         3
I2         0
I3         0
I4         0
I6         0
I7         0
I11        0
I15        0
I16        0
I17        0
I18        0
I19        0
I20        0
X1_m1      0
X2_m1      0
X3_m1      0
X4_m1      0
X5_m1      0
X6_m1      0
X7_m1      0
X8_m1      0
B1_m1      0
B2_m1      0
B3_m1      0
B4_m1      0
B5_m1      0
C1_m1      0
C2_m1      0
X1_m2      0
X2_m2      0
X3_m2      0
X4_m2      0
X5_m2      0
X6_m2      0
X7_m2      0
X8_m2      0
B1_m2      0
B2_m2      0
B3_m2      0
B4_m2      0
B5_m2      0
C1_m2      0
C2_m2      0
X1_m3      0
X2_m3      0
X3_m3      0
X4_m3      0
X5_m3      0
X6_m3      0
X7_m3      0
X8_m3      0
B1_m3      0
B2_m3      0
B3_m3      0
B4_m3      0
B5_m3      0
B6_m3      0
B7_m3      0
C1_m3      0
C2_m3      0
군인         0
농축업        0
무직         0

##### 전부 I1열에만 결측치가 있다. 일단 모든 결측치를 0.5로 대체하자.

In [9]:
pd.reset_option('max_rows')
train_fil = train.fillna(0.5)
test_fil = test.fillna(0.5)
valid_fil = valid.fillna(0.5)

### 데이터 label 변경

In [18]:
train_rep = train_fil
test_rep = test_fil
valid_rep = valid_fil
train_rep['label'].replace(-1,2,inplace=True)
test_rep['label'].replace(-1,2,inplace=True)
valid_rep['label'].replace(-1,2,inplace=True)

##### data split

In [19]:
x_train, y_train = train_rep.drop(['label','cust_no'], axis=1), train_rep["label"]
x_valid, y_valid = valid_rep.drop(['label','cust_no'],axis=1), valid_rep['label']
x_test, y_test = test_rep.drop(['label','cust_no'], axis=1), test_rep['label']

### 모델

In [20]:
model = xgb.XGBClassifier()
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [23]:
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 73.02%
